In [ ]:
"""
유전 알고리즘은 어떤 미지의 함수 Y = f(x)를 최적화하는 해 x를 찾기 위해, 진화를 모방한(Simulated evolution) 
탐색 알고리즘이라고 말할 수 있다.
유전 알고리즘은 특정한 문제를 풀기 위한 알고리즘이라기 보다는 문제를 풀기 위한 접근방법에 가까우며,
유전 알고리즘에서 사용할 수 있는 형식으로 바꾸어 표현할 수 있는 모든 문제에 대해서 적용할 수 있다.
일반적으로 문제가 계산 불가능할 정도로 지나치게 복잡할 경우 유전 알고리즘을 통하여, 
실제 최적해를 구하지는 못하더라도 최적해에 가까운 답을 얻기 위한 방안으로써 접근할 수 있다. 
이 경우 해당 문제를 푸는 데 최적화되어 있는 알고리즘보다 좋은 성능을 보여주지는 못하지만, 
대부분 받아들일 수 있는 수준의 해를 보여준다.

교차점이 하나! 싱글 크로스오버
교차점이 많다고 해서 좋은 해를 얻는 것은 아니고, 다양성이 많아지는 것뿐이다.
그러나 좋은 해를 얻을 때도 있다.

유전알고리즘 적합도 평가하는 방법!
코스트 함수를 사용한다. 코스트 함수를 설계하는 부분이 모델의 성능을 결정하고 가장 오래걸리는 부분이다.
모든 것을 100%만족시킬 수 없음을 유의한다.
"""

In [1]:
# LIST 주의할 점
# list1 = [0, 1, 2, 3, 4]가 있다고 가정. index는 순서대로 0, 1, 2, 3, 4이다.
# 여기서 del list1[2]를 한다고하면 [0, 1, 3, 4]가 된다. 여기서 index는 순서대로 0, 1, 2, 3이라는 것을 유의한다.
# 즉, 해당 인덱스의 데이터가 삭제된다고 해서 해당 데이터의 인덱스가 함께 삭제되는 것이 아님을 유의한다. (제일 큰 인덱스가 삭제됨)
# 이것은 np.array도 마찬가지다.

In [15]:
domain

[(0, 17),
 (0, 16),
 (0, 15),
 (0, 14),
 (0, 13),
 (0, 12),
 (0, 11),
 (0, 10),
 (0, 9),
 (0, 8),
 (0, 7),
 (0, 6),
 (0, 5),
 (0, 4),
 (0, 3),
 (0, 2),
 (0, 1),
 (0, 0)]

In [12]:
# -*- coding: utf-8 -*-
#writer : Tomas 

import time
import random
import math
import scipy.stats
import numpy as np

# 구간과 도메인의 정의

# 시간대 정의 6가지
times = ['A','B','C','D','E','F']

# 시간대 별로 3명씩 들어간다. 즉, 총 6x3=18 자리가 생긴다.
# 각 사람에게 배정할 수 있는 자리의 범위를 domain변수에 저장한다.
domain=[(0,(len(times)*3)-i-1) for i in range(0,len(times)*3)]

# cost function설계를 위한 제약조건 정리
# 특정 사람에 대한 좋아하는 시간대 정보와 싫어하는 사람 정보
prefs={
       1 : ['B', 'C' , 'A'],
       7 : ['E', 'F'],
       11 : ['A', 'B'],
       15 : ['E', 'D']
       }
        
hates = {
        13 : [1, 7], 
        10 : [11, 15],
        3 : [8, 12, 10]
        }

# 남자와 여자 사람정보
man = [range(0,12)] # 12명
woman = [range(12,18)] # 6명

# DNA구조. 결과값, 각 사람에게 배정된 시간대 정보
#vec = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

# Cost Function 구현
def dormcost(vec):
    
    # 밑의 규칙들을 모두 충족시키는 알고리즘 구현
    #선호하는 시간대에 따라 점수부여 (선호하는 시간대에 배정될 경우 코스트를 소량, 아닐경우 중량 증가)
    #싫어하는 사람에 따라 점수부여 (싫어하는 사람이 있을 경우만 코스트 증가)
    #각 시간대에 적어도 1명은 남자
    #C시간대에 여자가 있으면 안됨 
    #각 시간대에 4명 이상 있으면 안됨
    
    cost=0

    # Create list a of slots
    # 한 시간대에 3명씩 배정을 할 예정
    slots=[]
    for i in range(len(times)): slots+=[i,i,i]
  
    # 해 vec에 따른 시간대 설정
    times_band = printsolution(vec)
  
    #선호하는 시간대에 따라 점수부여 
    for i in prefs.keys():
        # 배정된 시간대 정보를 time에 저장 ex) 'A'
        time=times_band[i]
        # 선호하는 시간대 정보를 pref에 저장 ex) ['B', 'C' , 'A']
        pref=prefs[i]
      
        satisfy = False
        for j in range(len(pref)):           
            if pref[j]==time:
                # 배정된 시간대가 선호하는 시간대일 경우, 코스트를 (j+1)*2만큼 증가 --> 소량
                cost+=((j+1)*2)
                satisfy = True
        if satisfy == False:
            # 배정된 시간대가 선호하는 시간대가 아닐 경우, 코스트를  ((j+1)*2+3)만큼 증가 --> 중량
            cost+=((j+1)*2+3)
          
    # 싫어하는 사람에 따라 점수부여 
    for i in hates.keys():
        # 싫어하는 사람이 있는 특정 사람(A)이 배정된 시간대 정보를 time에 저장 ex) 'A'
        time=times_band[i]
        # 그 싫어하는 사람(B) 리스트를 ha에 저장
        ha=hates[i]
      
        for j in range(len(ha)):
            # 싫어하는 사람(B)이 배정된 시간대를 time_y에 저장
            y = int(vec[ha[j]])
            time_y=times[slots[y]]
            if time_y==time:
                # 싫어하는 사람이 있는 특정 사람(A)이 싫어하는 사람(B)과 같은 시간대에 배정될 경우, 코스트를 (7-j)만큼 증가
                cost+=(7-j)
              

    # 각 시간대별 적어도 1명은 남자가 있어야 함  
    idx = 0 
    time_dic = {'A':0 , 'B':0 , 'C':0 , 'D':0 , 'E':0 , 'F':0} 
    
    for t in times_band:
        if idx >=0 and idx < 12: # 남자가 12명 있음, 시간대별로 남자가 몇 명이 있는지 카운트
            time_dic[t] += 1
        idx += 1
       
    # C시간대에 여자가 있으면 안됨 
    idx = 0 
      
    for t in times_band:
        # idx가 12부터 여자가 배정된 시간대(t)이다.
        if idx >=12:
            time_dic[t] +=1
            # 여자가 C에 배정된 경우 패널티를 10점씩 부과한다.
            if t == 'C': cost += 10
        idx += 1
    
    # 각 시간대별로 4명 이상이 있으면 패널티를 100점씩 부과한다. (패널티를 높게 주는 이유는 한 시간대에 4명 이상 되는게 가장 싫기 때문)
    # 남자가 0명인 시간대 하나 당 패널티를 2점씩 부과해준다. 
    for i in time_dic.values():
        if i == 0:
            cost += 2
        if i >= 4:
            cost += 100
        
    # 다음과 같이 모든 코스트의 결과 값을 반환
    return cost
  

def printsolution(vec):
    slots = []
    times_band = []
  
    # 슬롯초기화 (각 시간대의 index값(3개씩)을 채움)
    for i in range(len(times)): slots+=[i,i,i]

    # 루프를 돌면서 전직원들의 시간배정
    for i in range(len(vec)):
        x=int(vec[i])
        # 직원의 시간대를 정한다. 
        time=times[slots[x]]
        
        #시간대를 추가함. 
        times_band.append(time)
    
    return times_band


# 위에서 정의한 함수들을 사용해서 유전자 알고리즘을 수행하는 함수

def geneticoptimize(domain,costf=dormcost,popsize=200,step=1, mutprob=0.5,elite=0.4,maxiter=20):
    
    # 변이 - 매개변수로 들어온 리스트 안의 값들 중 랜덤한 한 개를 step만큼 랜덤 변화를 주도록 함
    def mutate(vec):
        i=random.randint(0,len(domain)-1)
        if random.random()<0.5 and vec[i]-step>domain[i][0]:
            return vec[0:i]+[vec[i]-step]+vec[i+1:] 
        elif vec[i]+step<domain[i][1]:
            return vec[0:i]+[vec[i]+step]+vec[i+1:]
        else:
            return vec
      
    # 교차연산 - 매개변수 r1과 r2값들을 랜덤하게 서로 교차한 결과값을 반환
    def crossover(r1,r2):
        i=random.randint(1,len(domain)-1)
        return r1[0:i]+r2[i:]

    
    
    # 초기의 random 해 설정
    pop=[]
    for i in range(popsize):
        vec=random.sample(range(len(domain)), len(domain))
        pop.append(vec)
      
    # 엘리트해의 개수의 설정
    topelite=int(elite*popsize)
  
  
    # 메인루프 - 변이와 교차를 통해 데이터를 늘리고 계속해서 cost연산을 통해 cost가 낮은 해를 찾는 루프 수행
    for i in range(maxiter):
        # pop안에 있는 값들을 하나하나 코스트함수를 대입한 결과를 scores에 저장
        scores=[(costf(v),v) for v in pop]
        # scores를 오름차순으로 정렬 후, 코스트가 낮게 잡힌순으로 해 결과값만을 ranked에 저장
        scores.sort()
        ranked=[v for (s,v) in scores]

        # 엘리트해의 갯수만큼 추출하여 pop에 저장
        pop=ranked[0:topelite]

        # 변이와 교차의 반복. popsize만큼. 
        while len(pop)<popsize:
            # pop데이터 크기가 popsize 초과가 될 때 까지 변이와 교차연산을 반복하여 pop데이터를 늘림
            if random.random()<mutprob:
                # 랜덤한 확률로 ranked의 랜덤한 값들 중 하나를 골라 변이를 읽으키고 pop안에 추가
                c=random.randint(0,topelite)
                pop.append(mutate(ranked[c]))
            else:
                # 변이가 일어나지 않았다면 교차연산 수행(싱글), ranked의 엘리트해 안에 속하는 랜덤한 값들중 2개를 뽑아 서로 교차연산 후 pop안에 추가
                c1=random.randint(0,topelite)
                c2=random.randint(0,topelite)
                pop.append(crossover(ranked[c1],ranked[c2]))

        # 현재까지의 최적해 출력 (한번의 루프안에서 나온 최종 로우 코스트 해)
        print (i,scores[0][0],'--->',scores[0][1] , '==>' , printsolution(scores[0][1]))

    # 최종 최적해 반환
    return scores[0][1]


In [13]:
geneticoptimize(domain) 

0 12 ---> [9, 7, 2, 6, 8, 4, 16, 14, 17, 13, 15, 1, 0, 10, 5, 11, 12, 3] ==> ['D', 'C', 'A', 'C', 'C', 'B', 'F', 'E', 'F', 'E', 'F', 'A', 'A', 'D', 'B', 'D', 'E', 'B']
1 12 ---> [9, 7, 2, 6, 8, 4, 16, 14, 17, 13, 15, 1, 0, 10, 5, 11, 12, 3] ==> ['D', 'C', 'A', 'C', 'C', 'B', 'F', 'E', 'F', 'E', 'F', 'A', 'A', 'D', 'B', 'D', 'E', 'B']
2 12 ---> [9, 7, 2, 6, 8, 4, 16, 14, 17, 13, 15, 1, 0, 10, 5, 11, 12, 3] ==> ['D', 'C', 'A', 'C', 'C', 'B', 'F', 'E', 'F', 'E', 'F', 'A', 'A', 'D', 'B', 'D', 'E', 'B']
3 12 ---> [9, 7, 1, 6, 8, 4, 16, 14, 17, 13, 15, 1, 0, 10, 5, 11, 12, 3] ==> ['D', 'C', 'A', 'C', 'C', 'B', 'F', 'E', 'F', 'E', 'F', 'A', 'A', 'D', 'B', 'D', 'E', 'B']
4 12 ---> [9, 7, 1, 6, 8, 4, 16, 14, 17, 13, 15, 1, 0, 10, 5, 11, 12, 3] ==> ['D', 'C', 'A', 'C', 'C', 'B', 'F', 'E', 'F', 'E', 'F', 'A', 'A', 'D', 'B', 'D', 'E', 'B']
5 12 ---> [9, 7, 1, 6, 8, 4, 16, 14, 17, 13, 15, 1, 0, 10, 5, 11, 12, 3] ==> ['D', 'C', 'A', 'C', 'C', 'B', 'F', 'E', 'F', 'E', 'F', 'A', 'A', 'D', 'B', 'D', 'E

[9, 6, 1, 6, 6, 3, 15, 14, 17, 13, 15, 1, 0, 10, 5, 11, 12, 3]

In [ ]:
"""
## 위에서 나온 결과 분석

남1 남2 남3 남4 남5 남6 남7 남8 남9 남10 남11 남12 여13 여14 여15 여16 여17 여18
'D','C','A','C','C','B','F','E','F','E', 'F', 'A', 'A', 'D', 'B', 'D', 'E', 'B'

남2 - A B C 선호
남8 - E F 선호
남12 - A B 선호
여16 = D E 선호

여14 - 남2 남8 싫어
남11 - 남12 여16싫어
남4 - 남9 여13 남11싫어


# 각 조건 성립 여부 확인해보자

# 선호하는 위치에 있는가?
남2 선호
남8 선호
남12 선호
여16 선호

# 싫어하는 사람과 같이 근무 하는가?
여14 D - 아니
남11 F - 아니
남4 C - 아니

# 각 시간대에 적어도 1명 남자가 있는가?
A 있음
B 있음
C 있음
D 있음
E 있음
F 있음

# C 시간대에 여자가 있는가?
없음

# 각 시간대에 4명이상 있는 시간대가 있는가?
없음

모든 조건 성립...
"""